In [1]:
# Based on https://www.edureka.co/blog/snake-game-with-pygame/
from collections import defaultdict
import pygame
import time
import random

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Snake:
    def __init__(self, agent=None):
        self.white = (255, 255, 255)
        self.yellow = (255, 255, 102)
        self.black = (0, 0, 0)
        self.red = (213, 50, 80)
        self.green = (0, 255, 0)
        self.limegreen = (50,205,50)
        self.blue = (50, 153, 213)
        self.steelblue = (70,130,180)
        
        self.block_size = 20
        self.block_size_half = 10
        self.framerate = 20
        
        self.dis_width = 600
        self.dis_height = 400
        
        self.walls = [[],
                      [(300,0),(300,20),(300,40),(300,60),(300,80),(300,100),(300,120),
                       (300,260),(300,280),(300,300),(300,320),(300,340),(300,360),(300,380)],
                      [(300,60),(300,80),(300,100),(300,120),(300,140),(300,160),(300,180),
                       (300,200),(300,220),(300,240),(300,260),(300,280),(300,300),(300,320),
                       (120,200),(140,200),(160,200),(180,200),(200,200),(220,200),(240,200),
                       (260,200),(280,200),(300,200),(320,200),(340,200),(360,200),(380,200),
                       (400,200),(420,200),(440,200),(460,200),(480,200)]]
        self.wall = self.walls[0]
        self.score = 0
        self.dis = None
        
        self.agent = agent
        
    def start(self):
        pygame.init()
        pygame.display.set_caption('Snake Game by Ash and Max')
        
        self.dis = pygame.display.set_mode((self.dis_width, self.dis_height))
        self.clock = pygame.time.Clock()
        self.font_style = pygame.font.SysFont("bahnschrift", 25)
        self.score_font = pygame.font.SysFont("comicsansms", 35)
        self.time_font = pygame.font.SysFont("comicsansms", 35)
        
        self.foodx = None
        self.foody = None
        
        self.restart()
        self.main()
    
    def display_score(self):
        self.dis.blit(self.score_font.render("Score: " + str(self.score), True, self.black), [0, 0])
    
    def display_time(self):
        self.dis.blit(self.score_font.render("Time: " + str(self.time_counter), True, self.black), [0, 35])
    
    def display_snake(self):
        for x in self.snake_List:
            pygame.draw.rect(self.dis, self.steelblue, [x[0], x[1], self.block_size, self.block_size])
            if x[0] == self.last_foodx and x[1] == self.last_foody:
                pygame.draw.circle(self.dis, self.steelblue, 
                                   [x[0]+self.block_size_half, x[1]+self.block_size_half], 
                                   int(self.block_size*0.8))
            pygame.draw.circle(self.dis, self.steelblue, 
                               [self.x1+self.block_size_half, self.y1+self.block_size_half], 
                               self.block_size_half+5)
            if self.y1_change > 0:
                pygame.draw.line(self.dis, self.white, 
                                 [self.x1+self.block_size_half-5, self.y1+self.block_size_half-2], 
                                 [self.x1+self.block_size_half-2, self.y1+self.block_size_half+4],2)
                pygame.draw.line(self.dis, self.white, 
                                 [self.x1+self.block_size_half+5, self.y1+self.block_size_half-2], 
                                 [self.x1+self.block_size_half+2, self.y1+self.block_size_half+4],2)
            elif self.y1_change < 0:
                pygame.draw.line(self.dis, self.white, 
                                 [self.x1+self.block_size_half-2, self.y1+self.block_size_half-4], 
                                 [self.x1+self.block_size_half-5, self.y1+self.block_size_half+2],2)
                pygame.draw.line(self.dis, self.white, 
                                 [self.x1+self.block_size_half+2, self.y1+self.block_size_half-4], 
                                 [self.x1+self.block_size_half+5, self.y1+self.block_size_half+2],2)
            if self.x1_change > 0:
                pygame.draw.line(self.dis, self.white, 
                                 [self.x1+self.block_size_half-2, self.y1+self.block_size_half-5], 
                                 [self.x1+self.block_size_half+4, self.y1+self.block_size_half-2],2)
                pygame.draw.line(self.dis, self.white, 
                                 [self.x1+self.block_size_half-2, self.y1+self.block_size_half+5], 
                                 [self.x1+self.block_size_half+4, self.y1+self.block_size_half+2],2)
            elif self.x1_change < 0:
                pygame.draw.line(self.dis, self.white, 
                                 [self.x1+self.block_size_half-4, self.y1+self.block_size_half-2], 
                                 [self.x1+self.block_size_half+2, self.y1+self.block_size_half-5],2)
                pygame.draw.line(self.dis, self.white, 
                                 [self.x1+self.block_size_half-4, self.y1+self.block_size_half+2], 
                                 [self.x1+self.block_size_half+2, self.y1+self.block_size_half+5],2)
                
    def display_food(self):              
        pygame.draw.circle(self.dis, self.red, 
                           [self.foodx+self.block_size_half, self.foody+self.block_size_half], 
                           self.block_size_half)
        
    def display_wall(self):
        for x in self.wall:
            pygame.draw.rect(self.dis, self.black, [x[0], x[1], self.block_size, self.block_size])
 
    def message(self, msg, color, x, y):
        mesg = self.font_style.render(msg, True, color)
        self.dis.blit(mesg, [x, y])
        
    def restart(self):
        self.game_over = False
        self.game_close = False
        self.choose_stage = True
 
        self.x1 = self.block_size*3
        self.y1 = self.block_size*3
        
        self.x1_change = 0
        self.y1_change = self.block_size
 
        self.snake_List = []
        self.Length_of_snake = 1
        self.time_counter = 0
 
        self.generate_food()
        self.dead_cause = ""
        
    def main(self):
        while True:
            if self.game_close:
                pygame.quit()
                quit()
                break
            if self.choose_stage:
                self.dis.fill(self.limegreen)
                self.message("Please pick the environment", self.red, 150, 133)
                self.message("1. Basic", self.red, 150, 163)
                self.message("2. Walled 1", self.red, 150, 193)
                self.message("3. Walled 2", self.red, 150, 223)
                
                for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_1:
                            self.wall = self.walls[0]
                            self.choose_stage = False
                        if event.key == pygame.K_2:
                            self.wall = self.walls[1]
                            self.choose_stage = False
                        if event.key == pygame.K_3:
                            self.wall = self.walls[2]
                            self.choose_stage = False
            elif self.game_over and not self.game_close:                
                self.dis.fill(self.limegreen)
                self.message("You Lost because you " + self.dead_cause, self.red, 150, 133)
                self.message("Press C-Play Again or Q-Quit", self.red, 150, 163)
                
                self.display_score()
 
                for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_q:
                            self.game_close = True
                        if event.key == pygame.K_c:
                            self.restart()
            else:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        self.game_over = True
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_LEFT and self.x1_change == 0:
                            self.x1_change = -self.block_size
                            self.y1_change = 0
                        elif event.key == pygame.K_RIGHT and self.x1_change == 0:
                            self.x1_change = self.block_size
                            self.y1_change = 0
                        elif event.key == pygame.K_UP and self.y1_change == 0:
                            self.y1_change = -self.block_size
                            self.x1_change = 0
                        elif event.key == pygame.K_DOWN and self.y1_change == 0:
                            self.y1_change = self.block_size
                            self.x1_change = 0
                if self.agent != None:
                    self.x1_change, self.y1_change = self.agent.nextstep((self.x1, self.y1),
                                                                         (self.x1_change, self.y1_change),
                                                                         self.snake_List,
                                                                         self.wall,
                                                                         (self.foodx, self.foody))
                valid = True
                new_x = self.x1 + self.x1_change 
                new_y = self.y1 + self.y1_change
    
                if new_x >= self.dis_width or new_x < 0 or \
                    new_y >= self.dis_height or new_y < 0:
                    valid = False
                    self.dead_cause = "hit the boundary"
                
                for x in self.wall:
                    if x[0] == new_x and x[1] == new_y:
                        valid = False
                        self.dead_cause = "hit the wall"
                
                for x in self.snake_List[:-1]:
                    if x[0] == new_x and x[1] == new_y:
                        valid = False
                        self.dead_cause = "bite yourself"
                    
                if valid:
                    self.x1 = new_x
                    self.y1 = new_y
                else:
                    self.game_over = True
                    
                snake_Head = [self.x1, self.y1]
                self.snake_List.append(snake_Head)
                if len(self.snake_List) > self.Length_of_snake:
                    del self.snake_List[0]

                self.score = self.Length_of_snake - 1
                
                self.dis.fill(self.limegreen)
                self.display_wall()

                if self.x1 == self.foodx and self.y1 == self.foody:
                    self.generate_food()
                    self.Length_of_snake += 1
            
                self.clock.tick(self.framerate)
                self.time_counter += 1
                
                self.display_snake()
                self.display_food()
                self.display_score()
                self.display_time()
                
            pygame.display.update()
                
    def generate_food(self):
        self.last_foodx = self.foodx
        self.last_foody = self.foody
        pool, pool_new = [0 for x in range(600)], []
        for x in self.snake_List:
            pool[int(x[0]/20 + 30*x[1]/20)] = -1
        for x in self.wall:
            pool[int(x[0]/20 + 30*x[1]/20)] = -1
        for i in range(600):
            if pool[i] != -1:
                pool_new.append(i)
        seed = pool_new[random.randrange(0, len(pool_new))]
        self.foodx = int(seed%30)*20
        self.foody = int(seed/30)*20




In [3]:
class naive_agent:
    def __init__(self):
        pass

    def nextstep(self, head, current_direction, body, wall, food):
        x, y = head
        cx, cy = current_direction
        fx, fy = food
        dx, dy = fx - x, fy - y
        if dx != 0:
            return (20 if dx > 0 else -20) , 0
        return 0, (20 if dy > 0 else -20)

In [4]:
class fancy_agent:
    def __init__(self):
        pass
    
    def is_valid(self, x, y, body, wall):
        if x<0 or y<0 or x>580 or y> 380:
#             print(x, y)
            return False
        if (x,y) in wall:
            return False
        if len(body)<=2:
            return True
        if [x,y] in body[len(body)-1]:
            return True
        if [x,y] in body:
            return False
        return True

    def nextstep(self, head, current_direction, body, wall, food):
        if wall is None:
            wall = []
        x, y = head
        cx, cy = current_direction
        mcx, mcy = -cx, -cy
#         print(wall)
        fx, fy = food
        dx, dy = fx - x, fy - y
        nx, ny = (20 if dx > 0 else -20), (20 if dy > 0 else -20)
        if mcy == 0:
            if self.is_valid(x+nx, y, body, wall) and dx !=0 and nx!=mcx:
                return nx, 0
            if self.is_valid(x, y+ny, body, wall):
                return 0, ny
            return 0, -ny
        if self.is_valid(x, y+ny, body, wall) and dy!=0 and  ny!=mcy:
            return 0, ny
        if self.is_valid(x+nx, y, body, wall):
            return nx, 0
        return -nx, 0

In [5]:
class fanc2_agent:
    def __init__(self):
        self.profiled = False
        pass
    
    def is_valid(self, x, y, body, wall):
        if x<0 or y<0 or x>580 or y> 380:
            return False
        if (x,y) in wall:
#             print("in wall")
            return False
        if len(body)<=2:
            return True
        sz = len(body)-1
        if x == body[len(body)-1][0] and y==body[len(body)-1][1]:
#             print("come on")
            return True
        if [x,y] in body[:sz]:
            return False
        return True

    def get_score(self, x, y, body, food, wall):
        valid = self.is_valid(x, y, body, wall)
        fx, fy = food
        if not valid:
            return 10000
        dx, dy = abs(fx - x), abs(fy - y)
        score = dx+dy
#         if self.is_near_wall(x,y,wall):
#             score -= 1000
#         if dx+dy==0:
#             return -500
    
        if len(wall)!=0:
            if self.at_hotspot(x,y):
                score-=100
        return score
    
    def is_near_wall(self,x,y,wall):
        states = [
            [x+20,y+0],[x+0,y+20],
            [x-20,y+0],[x+0,y-20]
        ]
        for state in states:
            if state in wall:
                return True
            
    def at_hotspot(self, x,y):
        states = [
            [20,0],[0,20],
            [-20,0],[0,-20]
        ]
        for s in states:
            if (x+s[0],y+s[1]) in self.corners:
#                 print("Tama")
                return True
    
    def wall_profile(self, wall):
        states = [
            [20,0],[0,20],
            [-20,0],[0,-20]
        ]
        h = {}
        h = defaultdict(lambda:0,h)
        for w in wall:
            for s in states:
                if (w[0]+s[0], w[1]+s[1]) in wall:
                    h[w]+=1
        hot = []
        for k1,k2 in h:
            if k1<=0 or k2<=0 or k1>=580 or k2>=380:
                continue
#             return False
#             print(h[(k1,k2)])
            if h[(k1,k2)]==1:
                hot.append((k1,k2))
        self.corners = hot
#         print(self.corners)
                  
    
    def k_steps(self, head, current_direction, body, wall, food, d, k):
        x, y = head
        cx, cy = current_direction
        score = self.get_score(x,y, body, food, wall)
        min_score = 100000
        if k == d:
            return 0, 0, score
#         if k==1:
#             print("valid: ",self.is_valid(x,y, body, wall))
        if not self.is_valid(x,y, body, wall):
            return cx,cy, 100000
        mcx, mcy = -cx, -cy
        fx, fy = food
        dx, dy = fx - x, fy - y
        if abs(dx) + abs(dy) ==0:
            return 0,0, 0
        states = [
            [20,0],[0,20],
            [-20,0],[0,-20]
        ]
#         if len(body)>1:
#             print([x+mcx, y+mcy], body[1])
#         print(body)
        child = cx, cy
        for state in states:
            bn = body[:len(body)-2]
            bn.append([x+state[0],y+state[1]])
            dod = True
            if len(body)>0:
                lx = body[len(body)-2][0]
                ly = body[len(body)-2][1]
                if (x+state[0] == lx and y+state[1]==ly) :
#                     print([state[0], state[1]], body[len(body)-2])
                    dod = False
#             if [mcx, mcy]==state:
#                 if len(body)>1:
#                     print([x+state[0], y+state[1]], body[1])
#                 next
#             print(dod)
            if dod:
                m, n, i = self.k_steps((x+state[0], y+state[1]), current_direction, body, wall, food, d, k+1)
#                 print("state,i : ", state, m, n, i)
                if k==0 and i==0:
                    return state[0], state[1], i
                if i < min_score:
                    min_score = i
                    cx = state[0]
                    cy = state[1]
        
        return cx, cy, score+min_score
        
    
    def nextstep(self, head, current_direction, body, wall, food):
        if wall is None:
            wall = []
        else:
            if not self.profiled:
                self.profiled = True
                self.wall_profile(wall)
#         input(food)
#         if len(body)>0:
#             print(head,body)
#         print(head)
        move = self.k_steps( head, current_direction, body, wall, food, 8, 0)
#         print("move-",move)
        return move[:2]

In [6]:
snake = Snake(fanc2_agent())
snake.start()

snake body - List of all position body is present.
cy, cy - tells the movement by -20 / 20


In [7]:
pot = (1,2,3)
print(pot[:2])
print(pot[2])

(1, 2)
3
